# Data Science Basics - Scikit Learn - Category Classifer

In [ ]:
import json
import random

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

### Load In Data

#### Data Class

In [ ]:
class Category:
    ELECTRONICS = "ELECTRONICS"
    BOOKS = "BOOKS"
    CLOTHING = "CLOTHING"
    GROCERY = "GROCERY"
    PATIO = "PATIO"


class Sentiment:
    POSITIVE = "POSITIVE"
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"


class Review:
    def __init__(self, category, text, score):
        self.category = category
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:  # Amazon review is a 4 or 5
            return Sentiment.POSITIVE


class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_x(self, vectorizer):
        return vectorizer.transform(self.get_text())

    def get_y(self):
        return [x.sentiment for x in self.reviews]

    def get_category(self):
        return [x.category for x in self.reviews]

    def evenly_distribute(self):
        negative = list(
            filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews)
        )
        positive = list(
            filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews)
        )
        positive_shrunk = positive[: len(negative)]
        print(len(positive_shrunk))
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        print(self.reviews[0])

#### Load in Data

In [ ]:
file_names = [
    "data/electronics_small.json",
    "data/books_small.json",
    "data/clothing_small.json",
    "data/grocery_small.json",
    "data/patio_small.json",
]

file_categories = [
    Category.ELECTRONICS,
    Category.BOOKS,
    Category.CLOTHING,
    Category.GROCERY,
    Category.PATIO,
]

reviews = []
for i in range(len(file_names)):
    file_name = file_names[i]
    category = file_categories[i]
    with open(file_name) as f:
        for line in f:
            review_json = json.loads(line)
            review = Review(category, review_json["reviewText"], review_json["overall"])
            reviews.append(review)

### Data Prep

In [ ]:
train, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(train)
# train_container.evenly_distribute()
test_container = ReviewContainer(test)
# test_container.evenly_distribute()

corpus = train_container.get_text()
# vectorizer = CountVectorizer(binary=True)
# vectorizer.fit(corpus)
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

train_x = train_container.get_x(vectorizer)
train_y = train_container.get_category()

test_x = test_container.get_x(vectorizer)
test_y = test_container.get_category()

### Classification

In [ ]:
from sklearn import svm

clf = svm.SVC(C=16, kernel="linear", gamma="auto")
clf.fit(train_x, train_y)

In [ ]:
test_set = ["great for my wedding", "loved it in my garden", "good computer"]
new_test = vectorizer.transform(test_set)

clf.predict(new_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

# print(train_x)
gnb.fit(train_x.todense(), train_y)
gnb.score(test_x.todense(), test_y)

#### Performance

In [ ]:
y_pred = clf.predict(test_x)

f1_score(test_y, y_pred, average=None)

# for i in range(len(y_pred)):
#     print(y_pred[i], test_y[i])

In [ ]:
clf.score(test_x, test_y)

#### Tuning (with grid search)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {"kernel": ("linear", "rbf"), "C": [0.1, 1, 8, 16, 32]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x, train_y)

In [ ]:
clf.score(test_x, test_y)

### Pickle Model

#### Save classifier

In [ ]:
import pickle

with open("./models/category_classifier.pkl", "wb") as f:
    pickle.dump(clf, f)

with open("./models/category_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

#### Load Classifier

In [ ]:
import pickle

with open("./models/category_classifier.pkl", "rb") as f:
    clf = pickle.load(f)

with open("./models/category_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

In [ ]:
test_set = ["very quick speeds", "loved the necklace", "bad"]
new_test = vectorizer.transform(test_set)

clf.predict(new_test)

### Confusion Matrix

In [ ]:
import matplotlib as plt
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(test_x)

labels = [
    Category.ELECTRONICS,
    Category.BOOKS,
    Category.CLOTHING,
    Category.GROCERY,
    Category.PATIO,
]

cm = confusion_matrix(test_y, y_pred, labels=labels)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)

sn.heatmap(df_cm, annot=True, fmt="d")